In [1]:
import numpy as np  
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy.io as scio 
import scipy.io as sio
import pandas as pd
from tf_utils import random_mini_batches
from tensorflow.python.framework import ops

# 原始版本目前准确率最高

In [ ]:
# 创建占位符时增加L2，占位符就是告诉你这里有一个空位置，但是还不知道数据的类型是什么。。
def create_placeholders(n_x):
    isTraining = tf.placeholder_with_default(True, shape=())
    # 输入数据的特征                    
    x_in = tf.placeholder(tf.float32, [None, n_x], name="x_in")
    # 图的拉普拉斯矩阵L1（像素维度）
    lap1 = tf.placeholder(tf.float32, [None, None], name="lap1")
    # 波段方向上的Laplacian L2（波段维度）
    lap2 = tf.placeholder(tf.float32, [n_x, n_x], name="lap2")
    return x_in, lap1, lap2, isTraining

#这里定义的是一个初始化参数的函数，代表创建神经网络模型中的可训练参数
def initialize_parameters(n_x):
    tf.set_random_seed(1)
    #在这里新传入一个变量用来代替不同的维度
    x_w1 = tf.get_variable("x_w1", [n_x, n_x], initializer=tf.contrib.layers.xavier_initializer(seed=1))
    x_b1 = tf.get_variable("x_b1", [n_x], initializer=tf.zeros_initializer())
    x_w2 = tf.get_variable("x_w2", [n_x, n_x], initializer=tf.contrib.layers.xavier_initializer(seed=1))
    x_b2 = tf.get_variable("x_b2", [n_x], initializer=tf.zeros_initializer())
    parameters = {
        "x_w1": x_w1,
        "x_b1": x_b1,
        "x_w2": x_w2,
        "x_b2": x_b2
    }
    return parameters

# 新的GCN_layer函数，实现L1 * X * L2 * W
def GCN_layer(x_in, L1, L2, weights):
    # x_in  : (N, n_x)
    # L1    : (N, N)
    # L2    : (n_x, n_x)
    # weights : (n_x, n_y)

    # 先对空间维度进行卷积: L1 * X_in     500* 220
    x_mid1 = tf.matmul(L1, x_in)         # (N, n_x)

    # 再对波段维度进行卷积: (L1 * X) * L2  220*220
    x_mid2 = tf.matmul(x_mid1, L2)       # (N, n_x)

    # 最后乘上权重矩阵: (L1 * X * L2) * W  220*220
    x_out = tf.matmul(x_mid2, weights)   # (N, n_y)

    return x_out

def mynetwork(x, parameters, Lap1, Lap2, isTraining, momentums=0.9):
    with tf.name_scope("x_layer_1"):
        x_z1_bn = tf.layers.batch_normalization(x, momentum=momentums, training=isTraining)
        x_z1 = GCN_layer(x_z1_bn, Lap1, Lap2, parameters['x_w1']) + parameters['x_b1']
        x_z1_bn = tf.layers.batch_normalization(x_z1, momentum=momentums, training=isTraining)
        x_a1 = tf.nn.relu(x_z1_bn)

    with tf.name_scope("x_layer_2"):
        x_z2_bn = tf.layers.batch_normalization(x_a1, momentum=momentums, training=isTraining)
        x_z2 = GCN_layer(x_z2_bn, Lap1, Lap2, parameters['x_w2']) + parameters['x_b2']
#                      500*128 500*500 220*220 
    # L1正则化
    l1_loss = tf.reduce_sum(tf.abs(parameters['x_w2']))
    return x_z2, l1_loss

def mynetwork_optimization(output, x_in, l1_loss, reg, learning_rate, global_step):
    with tf.name_scope("cost"):
        reconstruction_loss = tf.reduce_mean(tf.square(output - x_in))
        cost = reconstruction_loss + reg * l1_loss
    with tf.name_scope("optimization"):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost, global_step=global_step)
    return cost, optimizer

def train_mynetwork(x_all, L_all, L2_all, learning_rate=0.001, beta_reg=0.001, num_epochs=200, print_cost=True):
    ops.reset_default_graph()
    (m, n_x) = x_all.shape
    costs = []
    
    #创建占位符
    x_in, lap1, lap2, isTraining = create_placeholders(n_x)
    #创建参数
    parameters = initialize_parameters(n_x)

    with tf.name_scope("network"):
        #前向传播
        output, l1_loss = mynetwork(x_in, parameters, lap1, lap2, isTraining)

    global_step = tf.Variable(0, trainable=False)
    with tf.name_scope("optimization"):
        cost, optimizer = mynetwork_optimization(output, x_in, l1_loss, beta_reg, learning_rate, global_step)

    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(num_epochs + 1):
            _ , epoch_cost = sess.run([optimizer, cost], feed_dict={x_in: x_all, lap1: L_all, lap2: L2_all, isTraining: True})
            if print_cost and epoch % 50 == 0:
                print("Epoch %i: Cost: %f" % (epoch, epoch_cost))
            if print_cost and epoch % 5 == 0:
                costs.append(epoch_cost)

        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per 5 epochs)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()
   
        band_scores = sess.run(parameters['x_w2'])
        print("Training completed!")

        return parameters, band_scores

In [3]:
import scipy.io as scio

# 加载数据
ALL_X = scio.loadmat('D:\\vscode\\HSI\\myGCN\\predata\\X.mat')['X']  # 去掉 .todense()
ALL_X = np.transpose(ALL_X)  # 或者用 ALL_X.T
ALL_L = scio.loadmat('D:\\vscode\\HSI\\myGCN\\predata\\LX.mat')['LX']  # 同样去掉 .todense()
ALL_L2 = scio.loadmat('D:\\vscode\\HSI\\myGCN\\predata\\LX2.mat')['LX2']  # 去掉 .todense()

print("ALL_X:", ALL_X.shape)
print("ALL_L:", ALL_L.shape)
print("ALL_L2:", ALL_L2.shape)


ALL_L = ALL_L.toarray()  # 转换为密集矩阵
ALL_L2 = ALL_L2.toarray()  # 转换为密集矩阵


# 
# ALL_X: (500, 220)
# ALL_L: (500, 500)
# ALL_L2: (220, 220)



ALL_X: (500, 220)
ALL_L: (500, 500)
ALL_L2: (220, 220)


In [4]:
parameters, band_scores = train_mynetwork(ALL_X, ALL_L, ALL_L2, learning_rate=0.001, beta_reg=0.001, num_epochs=200, print_cost=True)

# 保存波段得分
sio.savemat('band_scores.mat', {'band_scores': band_scores})

ValueError: Dimensions must be equal, but are 128 and 220 for 'network/x_layer_2/MatMul_1' (op: 'MatMul') with input shapes: [?,128], [220,220].

In [ ]:
# 选择得分最高的K个波段
K = 50
band_importance = np.sum(np.abs(band_scores), axis=0)
#为什么要相加
top_k_indices = np.argsort(band_importance)[-K:]
selected_bands = ALL_X[:, top_k_indices]

# 保存选择的波段
sio.savemat('selected_bands.mat', {'selected_bands': selected_bands})

# 打印选择的波段
print("选择的波段索引：", top_k_indices)
print("选择的波段数据：", selected_bands)


选择的波段索引： [176 141 148 182  94   3  18   5 194  47 208  32 107  91  14 101  66 175
  30 173  36 106 108  49 192 198  29 217 149 212  13 110  62 128  73 104
   4 189  92  68 159 134  42 197  46 136 209 112  33  57]
选择的波段数据： [[ 0.04726335  0.03636783  0.05962943 ...  0.0567863   0.04616272
   0.03840406]
 [ 0.04530458  0.03132842  0.03912318 ...  0.04719875 -0.00559831
  -0.06138576]
 [ 0.01762298  0.00969087  0.01321147 ...  0.01650228 -0.06273087
   0.05633597]
 ...
 [ 0.05557523 -0.07684766 -0.08114755 ... -0.10135758 -0.02720143
   0.0433742 ]
 [-0.02985972 -0.00970482 -0.01138071 ... -0.03293665  0.0673983
  -0.03845708]
 [ 0.02250978  0.01982882  0.01583971 ...  0.05139302 -0.0441799
  -0.0208163 ]]
